# Построение baseline-решений
В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:
1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:
1. Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
2. Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
3. Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

В качестве решения приложите получившийся jupyther notebook. Убедитесь, что в нем присутствуют:
- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyther notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

seed = 1903
first_categorial_index = 190

Загрузим train dataset, выделенный на предыдущей неделе

In [2]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",")
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv")
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


## Общая предобработка признаков

In [3]:
def scale_frame(frame):
    """Функция масштабирает frame на отрезок [0;1]"""
    scaler = StandardScaler()
    scaled_matrix = scaler.fit_transform(frame.as_matrix())
    return pd.DataFrame(scaled_matrix, columns=frame.columns)

def split_frame(frame, n_columns, c_columns):
    """Функция разбивает фрейм на два числовой и категориальный, а также масштабирует значения и заполняет пропуски."""
    n_frame = frame[n_columns].copy()
    c_frame = frame[c_columns].copy()
    # Посчитаем средние по колонкам
    numeric_means = n_frame.mean(axis=0, skipna=True)
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_means = numeric_means.dropna()
    dropped_numeric_colums = n_frame.columns.drop(numeric_means.index)
    n_frame = n_frame[list(numeric_means.index)]
    # Заполним пропущенные численные значения средними
    n_frame = n_frame.fillna(numeric_means, axis=0)
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    c_frame = c_frame.fillna("NaV")
    # Посчитаем количества уникальных значений по колонкам
    cat_unique_counts = c_frame.nunique()
    num_unique_counts = n_frame.nunique()
    # Удалим колонки с одним уникальным значением
    cat_columns_to_drop = cat_unique_counts[cat_unique_counts == 1].index
    num_columns_to_drop = num_unique_counts[num_unique_counts == 1].index
    c_frame = c_frame.drop(columns=cat_columns_to_drop)
    n_frame = n_frame.drop(columns=num_columns_to_drop)
    
    dropped_columns = np.concatenate([
        dropped_numeric_colums,
        list(num_columns_to_drop),
        list(cat_columns_to_drop)])
    return (n_frame, c_frame, dropped_columns)

Разделим коллекции на группы - числовые и категориальные.
Выполним начальные преобразования, которые мы делали на 1-й неделе
- заполним пропущенные числовые значения средними по колонке
- заполним пропущенные категориальные значения строками "NaV" (Not a Value)
- удалим колонки без значений и с только одним уникальным значением, т.к. они не сыграют роли в модели

Все эти действия производятся в функции split_frame (см. выше)

In [4]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

numeric_frame, categorial_frame, dropped_columns = split_frame(
    churn_data_frame,
    numeric_columns,
    categorial_columns)

Масштабируем числовые признаки

In [5]:
numeric_frame_scaled = scale_frame(numeric_frame)

Закодируем категориальные признаки значениями от 0 до n. Это в любом случае понадобится, т.к. например OneHotEncoder принимает на вход целочисленную матрицу.

In [6]:
def label_encode(X_categorial):
    """ Функция кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    X_num = np.empty(X_categorial.shape)
    for column_number in range(X_categorial.shape[1]):
        labelEncoder = LabelEncoder()
        column = X_categorial[:,column_number]
        for idx, val in enumerate(column):
            if(not(isinstance(val, str))):
                column[idx] = "NaV"
        num_column = labelEncoder.fit_transform(column)
        for row_number, val in enumerate(num_column):
            X_num[row_number, column_number] = val
    return X_num

In [7]:
categorial_frame_labeled = pd.DataFrame(
    label_encode(categorial_frame.as_matrix()),
    columns=categorial_frame.columns)

## Организация оценки качества

По предсказанным значениям и ответам функция строит матрицу ошибок. По полученной матрице функция считает precision и recall, затем по полученным precision и recall функция считает объединенную метрику, - F-меру.
На выходе функция возвращает кортеж значений в следующем порядке: F-мера, precision, recall, матрица ошибок

In [8]:
def fscore_precision_recall(y_predicted, y_test):
    """ Функция считает f-меру, precision и recall """
    true_positive_rate = 0
    false_positive_rate = 0
    false_negative_rate = 0
    true_negative_rate = 0
    for yp, yt in zip(y_predicted, y_test):
        if yp == 1 and yt == 1:
            true_positive_rate += 1
        elif yp == 1 and yt == -1:
            false_positive_rate += 1
        elif yp == -1 and yt == 1:
            false_negative_rate += 1
        else:
            true_negative_rate += 1
    precision = float(true_positive_rate)/float(true_positive_rate+false_positive_rate) if true_positive_rate > 0 else 0
    recall = float(true_positive_rate)/float(true_positive_rate+false_negative_rate) if true_positive_rate > 0 else 0
    fscore = 2*precision*recall/(precision+recall) if precision != 0 and recall != 0 else 0
    return (fscore, precision, recall, [[true_positive_rate, false_positive_rate], [false_negative_rate, true_negative_rate]])

Функция ниже разбивает выборку на несколько стратифицированных фолдов. После этого по каждому из фолдов строит модель, используя model_factory и обучает её. На полученной обученной модели функция строит предсказания и по ним считает F-меру.
Функция возвращает усредненное по всем фолдам значение F-меры.

In [17]:
def stratifiedKFold_fscore(X, y, model_factory, seed, folds_count = 3):
    """ Функция разбивает набор данных на folds_count, считает F-меру на каждом фолде и возвращает усредненное по фолдам значение. """
    skf = StratifiedKFold(n_splits=folds_count, shuffle=True, random_state=seed)
    f_score_sum = 0
    best_model = None
    best_score = 0
    best_precision = 0
    best_recall = 0
    best_table = None
    for train_indices, test_indices in skf.split(X, y):
        X_train = X.tocsc()[train_indices]
        X_test = X.tocsc()[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]
        model = model_factory()
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test)
        fscore, precision, recall, table = fscore_precision_recall(y_predicted, y_test)
        if(best_model is None or best_score < fscore):
            best_score = fscore
            best_model = model
            best_precision = precision
            best_recall = recall
            best_table = table
        f_score_sum += fscore
    return (float(f_score_sum)/float(folds_count), best_model, best_score, best_precision, best_recall, best_table)

## Построение baseline моделей
Для обработки категориальных признаков при построении baseline модели будем использовать OneHotEncoder и LabelEncoder.
Эти методы крайне просты. Другие методы обработки категориальных признаков несколько более сложны и не укладываются в рамки построения baseline модели.

Закодируем категориальныее признаки с помощью OneHotEncoder и объединим матрицы вещественных и бинарных признаков.

In [18]:
one_hot = OneHotEncoder()
cat_mtx = one_hot.fit_transform(categorial_frame_labeled.as_matrix())
X_one_hot = hstack([coo_matrix(numeric_frame_scaled.as_matrix()), cat_mtx])
y = churn_labels_frame.as_matrix().flatten()

Линейная модель

In [19]:
ridge_one_hot = stratifiedKFold_fscore(X_one_hot, y, RidgeClassifier, seed)
ridge_one_hot[0]

0.0009564801530368244

Случайный лес

In [20]:
random_forest_one_hot = stratifiedKFold_fscore(X_one_hot, y, RandomForestClassifier, seed)
random_forest_one_hot[0]

0.005711591898162834

Бустинг

In [21]:
gradient_boosting_one_hot = stratifiedKFold_fscore(X_one_hot, y, GradientBoostingClassifier, seed)
gradient_boosting_one_hot[0]

0.013149692641465175

Объединим матрицы вещественных и категориальных признаков, закодированных целыми числами

In [22]:
X_labeled = coo_matrix(pd.concat([numeric_frame_scaled, categorial_frame_labeled], axis=1, ignore_index=False).as_matrix())

Случайный лес

In [23]:
random_forest_labeled = stratifiedKFold_fscore(X_labeled, y, RandomForestClassifier, seed)
random_forest_labeled[0]

0.010393896575751042

Бустинг

In [24]:
gradient_boosting_labeled = stratifiedKFold_fscore(X_labeled, y, GradientBoostingClassifier, seed)
gradient_boosting_labeled[0]

0.017691092576074503

Линейная модель

In [25]:
ridge_labeled = stratifiedKFold_fscore(X_labeled, y, RidgeClassifier, seed)
ridge_labeled[0]

0.0

Сравним получившиеся оценки объединив их в таблицу

In [30]:
score_table = pd.DataFrame(
    [
        [ridge_one_hot[0], ridge_labeled[0], ridge_one_hot[2], ridge_labeled[2]],
        [random_forest_one_hot[0], random_forest_labeled[0], random_forest_one_hot[2], random_forest_labeled[2]],
        [gradient_boosting_one_hot[0], gradient_boosting_labeled[0], gradient_boosting_one_hot[2], gradient_boosting_labeled[2]]
    ],
    index=["Ridge", "Random forest", "Gradient boosting"],
    columns=["One hot mean", "Labeled mean", "One hot best", "Labeled best"])
score_table

,One hot mean,Labeled mean,One hot best,Labeled best
Ridge,0.000956,0.000000,0.002869,0.000000
Random forest,0.005712,0.010394,0.014265,0.014144
Gradient boosting,0.013150,0.017691,0.016854,0.028011
